In [ ]:
! pip install -q sentence-transformers datasets pylate beir ranx

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.1/134.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.6/340.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 100.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.3/99.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
 

In [ ]:
import wandb
wandb.init(mode="disabled")

In [ ]:
! rm -r output/
! rm -r pylate-index/

rm: cannot remove 'output/': No such file or directory
rm: cannot remove 'pylate-index/': No such file or directory


In [ ]:
import torch
from datasets import load_dataset
from sentence_transformers import (
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
)

from pylate import evaluation, losses, models, utils

# Define model parameters for contrastive training
model_name = "rasyosef/bert-medium-amharic"  # Choose the pre-trained model you want to use as base
batch_size = 32  # Larger batch size often improves results, but requires more memory

num_train_epochs = 4  # Adjust based on your requirements
# Set the run name for logging and output directory
run_name = "colbert-medium-amharic"
output_dir = f"output/{run_name}"

# 1. Here we define our ColBERT model. If not a ColBERT model, will add a linear layer to the base encoder.
model = models.ColBERT(
  model_name_or_path=model_name,
  document_length=256
)

# Compiling the model makes the training faster
# model = torch.compile(model)

In [ ]:
model.query_length, model.document_length

(32, 256)

In [ ]:
# Load Dataset
from datasets import load_dataset

am_relevance_dataset = load_dataset("rasyosef/amharic-passage-retrieval-dataset-with-negatives", split="train")#.select(range(4_000))
am_relevance_dataset = am_relevance_dataset.class_encode_column("category")
am_relevance_dataset = am_relevance_dataset.train_test_split(test_size=0.1, seed=16, stratify_by_column="category")
am_relevance_dataset

In [ ]:
test_passage_ids = set(am_relevance_dataset["test"]["passage_id"])
len(test_passage_ids)

4471

In [ ]:
from datasets import Dataset
from tqdm import tqdm
import random

ds_rows = []
for row in tqdm(am_relevance_dataset["train"]):
  neg_passages = row["negative_passages"]
  neg_passages = list(filter(lambda x: x["passage_id"] not in test_passage_ids, neg_passages))
  neg_passages_filtered = neg_passages[:4] + neg_passages[-4:]

  ds_rows.append({
      "query_id": row["query_id"],
      "passage_id": row["passage_id"],
      "query": row["query"],
      "positive": row["passage"],
      "negative_1": neg_passages_filtered[0]["passage"],
      "negative_2": neg_passages_filtered[2]["passage"],
      "negative_3": neg_passages_filtered[4]["passage"],
      "negative_4": neg_passages_filtered[6]["passage"],
    })

  ds_rows.append({
      "query_id": row["query_id"],
      "passage_id": row["passage_id"],
      "query": row["query"],
      "positive": row["passage"],
      "negative_1": neg_passages_filtered[1]["passage"],
      "negative_2": neg_passages_filtered[3]["passage"],
      "negative_3": neg_passages_filtered[5]["passage"],
      "negative_4": neg_passages_filtered[7]["passage"],
    })
  # print(ds_rows)
  # break

relevance_dataset = Dataset.from_list(ds_rows).sort("query_id")#.select(range(4000))
relevance_dataset

100%|██████████| 40237/40237 [00:18<00:00, 2164.88it/s]


Dataset({
    features: ['query_id', 'passage_id', 'query', 'positive', 'negative_1', 'negative_2', 'negative_3', 'negative_4'],
    num_rows: 80474
})

In [ ]:
len(test_passage_ids.intersection(relevance_dataset["passage_id"]))

0

In [ ]:
# Split the dataset (this dataset does not have a validation set, so we split the training set)

EVAL_SIZE = 4_000

eval_dataset = relevance_dataset.select(range(EVAL_SIZE))
train_dataset = relevance_dataset.select(range(EVAL_SIZE, len(relevance_dataset)))

train_dataset[0], train_dataset

({'query_id': '0ca6be67788e87a3c1d4719a9b75fbac',
  'passage_id': '84e77970cceefd0e0f5ee539e4064239',
  'query': 'የጋሞ ዞን ሕዝቦች ዘመን መለወጫ ዮ ማስቃላ በዓልን በዩኔስኮ ለማስመዝገብ እየተሰራ መሆኑ ተገለፀ',
  'positive': 'አዲስ አበባ፣ መስከረም 12፣ 2013 (ኤፍ.ቢ.ሲ) የጋሞ ዞን ሕዝቦች ዘመን መለወጫ ዮ ማስቃላ በዓልን በዩኔስኮ ለማስመዝገብ እየተሰራ መሆኑ ተገለጸ፡፡የዞኑ ባህል ቱሪዝም እና ስፖርት መምሪያ ኃላፊ አቶ ዘነበ በየነ እንዳሉት የጋሞ ሕዝቦች ማስቃላ በዓል በአለም ቱሪስቶች የሚጎበኝ ለማድረግ በመንከባከብ በዩኔስኮ ለማስመዝገብ ስራዎች እየተሰራ መሆኑን ተናግረዋል፡፡በዚህ ዓመት ከፍተኛ የቱሪስት ፍሰት መኖሩን የተናገሩት አቶ ዘነበ ይህም የሆነው በጋሞ አባቶች ጥረት ነው ብለዋል፡፡በዚህም የጋሞ ዞን ሰላም በመሆኑ ከቱሪዝም ዘርፍ ከ98 ሚሊየን ብር በላይ መገኘቱን ከዞኑ መንግስት ኮሚኒኬሽን መረጃ ያገኘነው መረጃ ያመላክታል፡፡የጋሞ ዞን ምክትል አስተዳዳሪ አቶ ብርሃኑ ዘውዴ በበኩላቸው ዞኑ የበርካታ የቱሪስት መስህቦች መገኛ መሆኑን ገልፀዋል።ሰላም ሲጠበቅ ባህል፣ ወጎች፣ የተፈጥሮ እና ሰው ሰራሽ መስህቦችን ለአለም ሕዝብ እንዲተዋወቅ እድል በመፍጠር ፤ዘርፉ ለዞኑ ኢኮኖሚኖሚ ማህበራዊ እድገት ላይ ከፍተኛ ድርሻ ያለው መሆኑንም ገልፀዋል፡፡',
  'negative_1': 'አዲስ አበባ፡- አገራዊ ባህላዊ እሴቶች ሳይበረዙና ሳይከለሱ ይበልጥ ማደበር፣ መጠበቅና ለቀጣይ ትውልድ ማስተላለፍ እንደሚገባ ተገለፀ፡፡የኢሬቻ በዓልን ምክንያት በማድረግ የተዘጋጀው ኤግዚቢሽን እና ባዛር ትናንት በኢግዚቢሽን\nማእከል በይፋ ተመርቆ በተከፈተበት ወቅት የኢፌዴሪ ባህልና ቱሪዝም ሚኒስትር ዶክተ

In [ ]:
eval_dataset[0], eval_dataset

({'query_id': '000020ad079f13ca77c92b6fa6ebfda5',
  'passage_id': 'ab51b3930d461d5743e6b13b01263f91',
  'query': '‹‹የአገር መከላከያ ሠራዊት ተዋጽኦ እንደ ተወካዮች ምክር ቤት መቀመጫ የሚከፋፈል አይደለም›› ጄኔራል ሳሞራ የኑስ',
  'positive': 'የአገር መከላከያ ሠራዊት ብሔራዊ ተዋጽኦ የጠበቀና የተመጣጠነ መሆን ያለበት ቢሆንም እንደ የተወካዮች ምክር ቤት መቀመጫ የሚከፋፈል እንዳልሆነ፣ የጦር ኃይሎች ጠቅላላ ኤታ ማዦር ሹም ጄኔራል ሳሞራ የኑስ ተናገሩ፡፡በአማራ ብሔራዊ ክልላዊ መንግሥት ዋና ከተማ በሆነችው በባህር ዳር ከተማ የተከበረውን ሦስተኛውን የሠራዊት ቀን ምክንያት በማድረግ በተዘጋጀው የውይይት መድረክ (ሲምፖዝየም)፣ ‹‹የኢፌዲሪ መከላከያ ሠራዊት ከየት ወዴት?›› በሚል ርዕስ ጥናታዊ ጽሑፍ ያቀረቡት ጄኔራል ሳሞራ የመከላከያ ሠራዊት አመጣጥ፣ ዕድገትና አሁን የሚገኝበትን ደረጃ ተንትነዋል፡፡ቀደም ሲል ደርግን ያሸነፈው የኢሕአዴግ ሠራዊት የአሁኑ መከላከያ ሠራዊት መሠረት እንደሆነ አስታውሰው፣ ለማመጣጠን ሲባል ከ30 ሺሕ በላይ ነባር ታጋዮች እንዲቀነሱ የተደረገበት ምክንያት ብሔራዊ አስተዋጽኦን ለማጠናከር እንደሆነ ገልጸዋል፡፡ ‹‹ለታጋዮችም ሆነ ለአመራሮች ፈታኝና ከባድ ውሳኔ የነበረ ቢሆንም ለሕዝብ ጥቅም ሲባል ወስነናል፤›› በማለት የገለጹት ጄኔራል ሳሞራ፣ በመከላከያ ሠራዊት አነስተኛ ተሳትፎ ከነበራቸው ብሔር ብሔረሰቦች አዲስ አባላት በመመልመል፣ ከወንጀል ነፃ የሆኑ ተፈላጊ ችሎታና ሙያ የነበራቸው የደርግ ሠራዊት አባላትም ታክለውበት ሠራዊቱ እንደ አዲስ መደራጀቱን አብራርተዋል፡፡ በወቅቱ ካልተቀነሱት የሕወሓት ታጋዮች ላይ ሁለት ማዕረግ እንደተቀነሰ፣ በአንፃሩ ደግሞ ለሌሎች ብሔ

In [ ]:
# Define the loss function
train_loss = losses.Contrastive(model=model)

# Initialize the evaluator
dev_evaluator = evaluation.ColBERTTripletEvaluator(
    anchors=eval_dataset["query"] * 4,
    positives=eval_dataset["positive"] * 4,
    negatives=eval_dataset["negative_1"] + eval_dataset["negative_2"] + eval_dataset["negative_3"] + eval_dataset["negative_4"],
)

In [ ]:
eval_steps = 1000

# Configure the training arguments (e.g., batch size, evaluation strategy, logging steps)
args = SentenceTransformerTrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=False,  # Set to True if you have a GPU that supports BF16
    run_name=run_name,  # Will be used in W&B if `wandb` is installed
    learning_rate=1e-5,
    lr_scheduler_type="linear",
    eval_strategy="steps",
    eval_steps=eval_steps,
    save_strategy="steps",
    save_steps=eval_steps,
    # save_total_limit=2,
    logging_strategy="steps",
    logging_steps=eval_steps,
)

# Initialize the trainer for the contrastive training
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=train_loss,
    evaluator=dev_evaluator,
    data_collator=utils.ColBERTCollator(model.tokenize),
)

In [ ]:
# Start the training process
trainer.train()

### **Evaluate**

In [ ]:
dev_evaluator(
    model=model,
    # model=models.ColBERT("/content/output/contrastive-bert-small/checkpoint-2850"),
    output_path="."
  )

{'accuracy': 0.975687563419342}

In [1]:
# PUSH MODEL TO HUGGINGFACE

# import os
# from google.colab import userdata

# os.environ["HF_TOKEN"] = userdata.get("HF_WRITE_TOKEN")

# # push model to hub
# trainer.model.push_to_hub("colbert-bert-amharic-medium-45k")

### **BIER Eval**

In [ ]:
from pylate import evaluation, indexes, models, retrieve

# Step 1: Initialize the ColBERT model

# dataset = "scifact" # Choose the dataset you want to evaluate
# model = models.ColBERT(
#     model_name_or_path="lightonai/GTE-ModernColBERT-v1",
#     device="cuda" # "cpu" or "cuda" or "mps"
# )

# Step 2: Create a Voyager index
index = indexes.Voyager(
    index_folder="pylate-index",
    index_name="colbert-eval",
    override=True,  # Overwrite any existing index
)

test_dataset = am_relevance_dataset["test"]

documents = [
    {"id": pid, "text": pos}
    for pid, pos in zip(
        am_relevance_dataset["test"]["passage_id"] + am_relevance_dataset["train"]["passage_id"],
        am_relevance_dataset["test"]["passage"] + am_relevance_dataset["train"]["passage"]
    )
  ]
queries = [q for q in test_dataset["query"]]
qrels = {query: {pid:1} for query, pid in zip(test_dataset["query"], test_dataset["passage_id"])}

# Step 4: Encode the documents
documents_embeddings = model.encode(
    [document["text"] for document in documents],
    batch_size=32,
    is_query=False,  # Indicate that these are documents
    show_progress_bar=True,
)

# Step 5: Add document embeddings to the index
index.add_documents(
    documents_ids=[document["id"] for document in documents],
    documents_embeddings=documents_embeddings,
)

# Step 6: Encode the queries
queries_embeddings = model.encode(
    queries,
    batch_size=32,
    is_query=True,  # Indicate that these are queries
    show_progress_bar=True,
)

# Step 7: Retrieve top-k documents
retriever = retrieve.ColBERT(index=index)
scores = retriever.retrieve(
    queries_embeddings=queries_embeddings,
    k=100,  # Retrieve the top 100 matches for each query
    batch_size=64,
)

# Step 8: Evaluate the retrieval results
results = evaluation.evaluate(
    scores=scores,
    qrels=qrels,
    queries=queries,
    metrics=[f"ndcg@{k}" for k in [5, 10, 100]] # NDCG for different k values
    + ["recall@5", "recall@10", "recall@50", "recall@100"]   # Recall at k
    + ["mrr@5", "mrr@10", "mrr@100"]
)

print(results)

In [ ]:
# Step 8: Evaluate the retrieval results
results = evaluation.evaluate(
    scores=scores,
    qrels=qrels,
    queries=queries,
    metrics=[f"ndcg@{k}" for k in [5, 10, 100]] # NDCG for different k values
    + ["recall@5", "recall@10", "recall@50", "recall@100"]   # Recall at k
    + ["mrr@5", "mrr@10", "mrr@100"]
)

print(results)

metrics = ["ndcg@5", "ndcg@10", "ndcg@100", "mrr@5", "mrr@10", "mrr@100", "recall@5", "recall@10", "recall@50", "recall@100"]
for m in metrics:
  print(f"{m}: {round(results[m], 3)}")

{'ndcg@5': np.float64(0.8221140429518673), 'ndcg@10': np.float64(0.8312976580329593), 'ndcg@100': np.float64(0.8435650833058462), 'recall@5': np.float64(0.8823529411764706), 'recall@10': np.float64(0.9107582196376649), 'recall@50': np.float64(0.9543726235741445), 'recall@100': np.float64(0.9675687765600537), 'mrr@5': np.float64(0.8017893088794452), 'mrr@10': np.float64(0.8055776556503463), 'mrr@100': np.float64(0.8081691817457954)}
ndcg@5: 0.822
ndcg@10: 0.831
ndcg@100: 0.844
mrr@5: 0.802
mrr@10: 0.806
mrr@100: 0.808
recall@5: 0.882
recall@10: 0.911
recall@50: 0.954
recall@100: 0.968


In [ ]:
queries_embeddings[0].shape

(32, 128)